# Cleaning Yfinance DFs
Don't forget to update requirements.txt if you install additional libraries

In [2]:
# #Uncomment and run the following command to update the requirements.txt file:
# pip freeze > requirements.txt
# !pip install -r ../requirements.txt
!pip install sqlalchemy psycopg2

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\georg.DESKTOP-2FS9VF1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import sys
import os
from Pipeline.pipeline import Pipeline
import pandas as pd


db_config = {
        "user": "ais_sp500_db_user",
        "password": "lFmNQ7l0yiXhSc1Bet3OHw8EuUeDuv7F",
        "host": "dpg-d0rflvndiees73c05ea0-a.oregon-postgres.render.com",
        "port": 5432,
        "database": "ais_sp500_db"
    }
pipeline = Pipeline(db_config)
sp500_info = pipeline.load_data("sp500_info")
print(sp500_info.head())
sp500_info.to_csv("sp500_info.csv", index=False)

Loading data from PostgreSQL table: sp500_info
Data loaded successfully from sp500_info (records: 503)
  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                GICS Sub-Industry    Headquarters Location  Date added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois  2012-12-31   
4  IT Consulting & Other Services          Dublin, Ireland  2011-07-06   

       CIK      Founded  
0    66740         1902  
1    91142         1916  
2     1800         1888  
3  15

In [4]:
# We don't need last 3 columns, so we can drop them
sp500_info = sp500_info.drop(columns=["Founded", "CIK", "Date added"])

# Next, for Headquarters Location split by ", " and take the last part
sp500_info["Headquarters Location"] = sp500_info["Headquarters Location"].apply(
    lambda x: x.split(", ")[-1] if isinstance(x, str) else x
)

sp500_info.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location
0,MMM,3M,Industrials,Industrial Conglomerates,Minnesota
1,AOS,A. O. Smith,Industrials,Building Products,Wisconsin
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,Illinois
3,ABBV,AbbVie,Health Care,Biotechnology,Illinois
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,Ireland


In [5]:
# Next, we'll have a look at sp500_income_statements
sp500_income_statements = pipeline.load_data("sp500_income_statements")
print(sp500_income_statements.head())
sp500_income_statements.to_csv("sp500_income_statements.csv", index=False)

Loading data from PostgreSQL table: sp500_income_statements
Data loaded successfully from sp500_income_statements (records: 9604)
      Period  Tax Effect Of Unusual Items  Tax Rate For Calcs  \
0 2024-12-31                  261188000.0               0.167   
1 2023-12-31                    9144000.0               0.254   
2 2022-12-31                  110385000.0               0.045   
3 2021-12-31                          0.0               0.178   
4 2020-12-31                          NaN                 NaN   

   Normalized EBITDA  Total Unusual Items  \
0       5.809000e+09         1.564000e+09   
1      -8.379000e+09         3.600000e+07   
2       4.044000e+09         2.453000e+09   
3       9.607000e+09         0.000000e+00   
4                NaN                  NaN   

   Total Unusual Items Excluding Goodwill  \
0                            1.564000e+09   
1                            3.600000e+07   
2                            2.453000e+09   
3                           

In [6]:
# load the csv file
import pandas as pd
sp500_income_statements = pd.read_csv("sp500_income_statements.csv")
# drop the following columns: 
# Restructuring And Mergern Acquisition	Depreciation Amortization Depletion Income Statement	Depreciation And Amortization In Income Statement	Amortization	Amortization Of Intangibles Income Statement	Otherunder Preferred Stock Dividend	Other Special Charges	Other Operating Expenses	Average Dilution Earnings	Selling And Marketing Expense	Earnings From Equity Interest	Gain On Sale Of Ppe	Write Off	Loss Adjustment Expense	Net Policyholder Benefits And Claims	Policyholder Benefits Gross	Policyholder Benefits Ceded	Provision For Doubtful Accounts	Depreciation Income Statement	Preferred Stock Dividends	Other Taxes	Excise Taxes	Occupancy And Equipment	Professional Expense And Contract Services Expense	Other Non Interest Expense	Total Other Finance Cost	Rent Expense Supplemental	Rent And Landing Fees	Insurance And Claims	Depletion Income Statement	Net Income From Tax Loss Carryforward	Net Income Extraordinary	Securities Amortization
columns_to_drop = [
    "Restructuring And Mergern Acquisition",
    "Depreciation Amortization Depletion Income Statement",
    "Depreciation And Amortization In Income Statement",
    "Amortization",
    "Amortization Of Intangibles Income Statement",
    "Otherunder Preferred Stock Dividend",
    "Other Special Charges",
    "Other Operating Expenses",
    "Average Dilution Earnings",
    "Selling And Marketing Expense",
    "Earnings From Equity Interest",
    "Gain On Sale Of Ppe",
    "Write Off",
    "Loss Adjustment Expense",
    "Net Policyholder Benefits And Claims",
    "Policyholder Benefits Gross",
    "Policyholder Benefits Ceded",
    "Provision For Doubtful Accounts",
    "Depreciation Income Statement",
    "Preferred Stock Dividends",
    "Other Taxes",
    "Excise Taxes",
    "Occupancy And Equipment",
    "Professional Expense And Contract Services Expense",
    "Other Non Interest Expense",
    "Total Other Finance Cost",
    "Rent Expense Supplemental",
    "Rent And Landing Fees",
    "Insurance And Claims",
    "Depletion Income Statement",
    "Net Income From Tax Loss Carryforward",
    "Net Income Extraordinary",
    "Securities Amortization"
]

sp500_income_statements = sp500_income_statements.drop(columns=columns_to_drop)
# drop rows with 10 or less values
sp500_income_statements = sp500_income_statements.dropna(thresh=10)



#save for observability
sp500_income_statements.to_csv("sp500_income_statements_cleaned.csv", index=False)

In [7]:
# Next, sp500_balance_sheets
sp500_balance_sheets = pipeline.load_data("sp500_balance_sheets")
print(sp500_balance_sheets.head())
sp500_balance_sheets.to_csv("sp500_balance_sheets.csv", index=False)

Loading data from PostgreSQL table: sp500_balance_sheets
Data loaded successfully from sp500_balance_sheets (records: 9828)
      Period  Treasury Shares Number  Ordinary Shares Number  Share Issued  \
0 2024-12-31             404562753.0             539470303.0   944033056.0   
1 2023-12-31             391451920.0             552581136.0   944033056.0   
2 2022-12-31             394787951.0             549245105.0   944033056.0   
3 2021-12-31             372187578.0             571845478.0   944033056.0   
4 2020-12-31                     NaN                     NaN           NaN   

       Net Debt    Total Debt  Tangible Book Value  Invested Capital  \
0  7.444000e+09  1.365900e+10        -3.649000e+09      1.688600e+10   
1  1.030000e+10  1.675100e+10        -2.898000e+09      2.084200e+10   
2  1.228400e+10  1.685500e+10        -2.767000e+09      3.066100e+10   
3  1.279900e+10  1.831000e+10        -3.728000e+09      3.240900e+10   
4           NaN           NaN                  

In [8]:
# drop the following columns:
#Foreign Currency Translation Adjustments	Minimum Pension Liabilities	Preferred Stock	Non Current Accrued Expenses	Long Term Provisions	Current Provisions	Inventories Adjustments Allowances	Other Payable	Dividends Payable	Other Investments	Available For Sale Securities	Other Equity Interest	Non Current Deferred Revenue	Leases	Interest Payable	Cash Equivalents	Cash Financial	Dueto Related Parties Current	Non Current Prepaid Assets	Duefrom Related Parties Current	Preferred Stock Equity	Preferred Securities Outside Stock Equity	Non Current Note Receivables	Investments In Other Ventures Under Equity Method	Investmentsin Associatesat Cost	Unrealized Gain Loss	Loans Receivable	Derivative Product Liabilities	Line Of Credit	Financial Assets	Current Deferred Assets	Taxes Receivable	Current Deferred Taxes Liabilities	Receivables Adjustments Allowances	Investmentsin Joint Venturesat Cost	Investment Properties	Notes Receivable	Preferred Shares Number	Other Inventories	Current Notes Payable	Held To Maturity Securities	Cash Cash Equivalents And Federal Funds Sold	Trading Securities	Financial Assets Designatedas Fair Value Through Profitor Loss 	Investmentsin Subsidiariesat Cost	Current Deferred Taxes Assets	Accrued Interest Receivable	Dueto Related Parties Non Current	Duefrom Related Parties Non Current	Restricted Common Stock	Total Partnership Capital	Limited Partnership Capital

sp500_balance_sheets = pd.read_csv("sp500_balance_sheets.csv")
columns_to_drop_balance = [
    "Foreign Currency Translation Adjustments",
    "Minimum Pension Liabilities",
    "Preferred Stock",
    "Non Current Accrued Expenses",
    "Long Term Provisions",
    "Current Provisions",
    "Inventories Adjustments Allowances",
    "Other Payable",
    "Dividends Payable",
    "Other Investments",
    "Available For Sale Securities",
    "Other Equity Interest",
    "Non Current Deferred Revenue",
    "Leases",
    "Interest Payable",
    "Cash Equivalents",
    "Cash Financial",
    "Dueto Related Parties Current",
    "Non Current Prepaid Assets",
    "Duefrom Related Parties Current",
    "Preferred Stock Equity",
    "Preferred Securities Outside Stock Equity",
    "Non Current Note Receivables",
    "Investments In Other Ventures Under Equity Method",
    "Investmentsin Associatesat Cost",
    "Unrealized Gain Loss",
    "Loans Receivable",
    "Derivative Product Liabilities",
    "Line Of Credit",
    "Financial Assets",
    "Current Deferred Assets",
    "Taxes Receivable",
    "Current Deferred Taxes Liabilities",
    "Receivables Adjustments Allowances",
    "Investmentsin Joint Venturesat Cost",
    "Investment Properties",
    "Notes Receivable",
    "Preferred Shares Number",
    "Other Inventories",
    "Current Notes Payable",
    "Held To Maturity Securities",
    "Cash Cash Equivalents And Federal Funds Sold",
    "Trading Securities",
    "Financial Assets Designatedas Fair Value Through Profitor Loss ",
    "Investmentsin Subsidiariesat Cost",
    "Current Deferred Taxes Assets",
    "Accrued Interest Receivable",
    "Dueto Related Parties Non Current",
    "Duefrom Related Parties Non Current",
    "Restricted Common Stock",
    "Total Partnership Capital", 
    "Limited Partnership Capital"
]

sp500_balance_sheets.drop(columns=columns_to_drop_balance, inplace=True)
# drop rows with 10 or less values
sp500_balance_sheets.dropna(thresh=10, inplace=True)


#save for observability
sp500_balance_sheets.to_csv("sp500_balance_sheets_cleaned.csv", index=False)

In [9]:
#Next for sp500_cashflow_statements

sp500_cashflow_statements = pipeline.load_data("sp500_cashflow_statements")
print(sp500_cashflow_statements.head())
sp500_cashflow_statements.to_csv("sp500_cashflow_statements.csv", index=False)

Loading data from PostgreSQL table: sp500_cashflow_statements
Data loaded successfully from sp500_cashflow_statements (records: 9872)
      Period  Free Cash Flow  Repurchase Of Capital Stock  Repayment Of Debt  \
0 2024-12-31    6.380000e+08                -1.801000e+09      -2.656000e+09   
1 2023-12-31    5.065000e+09                -3.300000e+07      -3.086000e+09   
2 2022-12-31    3.842000e+09                -1.464000e+09      -1.179000e+09   
3 2021-12-31    5.851000e+09                -2.199000e+09      -1.144000e+09   
4 2020-12-31             NaN                          NaN                NaN   

   Issuance Of Debt  Capital Expenditure  Interest Paid Supplemental Data  \
0      8.367000e+09        -1.181000e+09                      505000000.0   
1      2.835000e+09        -1.615000e+09                      520000000.0   
2      1.000000e+06        -1.749000e+09                      440000000.0   
3      1.000000e+06        -1.603000e+09                      472000000.0   


In [10]:
#load the csv file
sp500_cashflow_statements = pd.read_csv("sp500_cashflow_statements.csv")
# drop the following columns:
text = "Gain Loss On Investment Securities	Short Term Debt Payments	Short Term Debt Issuance	Provisionand Write Offof Assets	Issuance Of Capital Stock	Common Stock Issuance	Change In Other Current Liabilities	Unrealized Gain Loss On Investment Securities	Gain Loss On Sale Of PPE	Earnings Losses From Equity Investments	Other Cash Adjustment Inside Changein Cash	Net Preferred Stock Issuance	Preferred Stock Issuance	Net Foreign Currency Exchange Gain Loss	Amortization Of Securities	Net Intangibles Purchase And Sale	Purchase Of Intangibles	Cash From Discontinued Financing Activities	Cash From Discontinued Investing Activities	Cash From Discontinued Operating Activities	Preferred Stock Dividend Paid	Dividend Received Cfo	Preferred Stock Payments	Net Investment Properties Purchase And Sale	Sale Of Investment Properties	Purchase Of Investment Properties	Other Cash Adjustment Outside Changein Cash	Change In Interest Payable	Dividends Received Cfi	Sale Of Intangibles	Excess Tax Benefit From Stock Based Compensation	Interest Received Cfi	Cash Flow From Discontinued Operation	Depletion	Interest Paid Cff	Taxes Refund Paid	Cash Flowsfromusedin Operating Activities Direct	Classesof Cash Payments	Other Cash Paymentsfrom Operating Activities	Paymentson Behalfof Employees	Classesof Cash Receiptsfrom Operating Activities	Other Cash Receiptsfrom Operating Activities	Dividend Paid Cfo	Interest Paid Cfo"
sp500_cashflow_statements.drop(columns=text.split("	"), inplace=True)
sp500_cashflow_statements.dropna(thresh=10, inplace=True)
sp500_cashflow_statements.to_csv("sp500_cashflow_statements_cln.csv")

In [11]:
#prices now
sp500_prices = pipeline.load_data("sp500_prices")
print(sp500_prices.head())
sp500_prices.to_csv("sp500_prices.csv")

Loading data from PostgreSQL table: sp500_prices
Data loaded successfully from sp500_prices (records: 476)
        Date   MMM_Close  AOS_Close  ABT_Close  ABBV_Close   ACN_Close  \
0 2020-01-02  121.761093  43.202148  78.908424   70.934235  193.477829   
1 2020-01-03  120.712570  42.822319  77.946465   70.260941  193.155594   
2 2020-01-06  120.827560  43.093632  78.354836   70.815414  191.894333   
3 2020-01-07  120.340561  42.804226  77.919243   70.411430  187.751328   
4 2020-01-08  122.187233  42.740917  78.236870   70.910477  188.119583   

   ADBE_Close  AMD_Close  AES_Close  AFL_Close  ...  WMB_Close   WTW_Close  \
0  334.429993  49.099998  16.533529  46.703022  ...  17.335360  190.150528   
1  331.809998  48.599998  16.343681  46.379131  ...  17.409159  190.197067   
2  333.709991  48.389999  16.533529  46.247818  ...  17.637936  190.429794   
3  333.390015  48.250000  16.591305  45.810112  ...  17.637936  190.038834   
4  337.869995  47.830002  16.558292  45.950176  ...  17.36

In [12]:
# record and drop columns with NaN
nan_columns = sp500_prices.columns[sp500_prices.isna().any()].tolist()
print("Columns with NaN values:", nan_columns)
sp500_prices.drop(columns=nan_columns, inplace=True)
# 


Columns with NaN values: ['ABNB_Close', 'APO_Close', 'BALL_Close', 'BRK.B_Close', 'BK_Close', 'BA_Close', 'BF.B_Close', 'BXP_Close', 'CARR_Close', 'CNC_Close', 'COIN_Close', 'CEG_Close', 'DG_Close', 'DASH_Close', 'ETN_Close', 'EPAM_Close', 'EXE_Close', 'XOM_Close', 'FOX_Close', 'GEHC_Close', 'GEV_Close', 'HST_Close', 'IR_Close', 'IFF_Close', 'KVUE_Close', 'KEYS_Close', 'KKR_Close', 'MAR_Close', 'NI_Close', 'ORLY_Close', 'ORCL_Close', 'OTIS_Close', 'PLTR_Close', 'PAYC_Close', 'PYPL_Close', 'PNW_Close', 'PPL_Close', 'PGR_Close', 'RL_Close', 'RF_Close', 'RCL_Close', 'SW_Close', 'SOLV_Close', 'TT_Close', 'UAL_Close', 'VLO_Close', 'VLTO_Close']


In [34]:
# So we can make 3 different functions to clean the data
def clean_sp500_dataframe(df: pd.DataFrame, columns: list, threshold:int):
    """Clean SP500 DF
    Input:
    df: either sp500_info or sp500 income statement/cashflow (not price)
    columns: list of columns to drop
    threshold: min amount of values per row to consider keeping
    """
    if "Symbol" in df.columns:
        df.rename(columns={"Symbol": "Ticker"}, inplace=True)
    df.drop(columns=columns,inplace=True)
    df.dropna(thresh=threshold, inplace= True)
    if "Headquarters Location" in df.columns:
        df["Headquarters Location"] = df["Headquarters Location"].apply(
            lambda x: x.split(", ")[-1] if isinstance(x, str) else x
        )
    
    df["Ticker"] = df["Ticker"].astype("category")
    if "Period" in df.columns:
        df["Period"] = pd.to_datetime(df["Period"], errors='coerce')

    return df

def clean_sp500_price_df(df):
    """
    To use for price based dataframe. Returns tuple with list of dropped columns (if applicable)
    Input: df: either sp500_prices or macro prices
    Output: (df, list of dropped cols)
    """
    nan_columns = df.columns[df.isna().any()].tolist()
    df.drop(columns=nan_columns, inplace=True) 
    return (df, nan_columns)



In [ ]:
sp500_income_statements = pd.read_csv("sp500_income_statements.csv")
income = clean_sp500_dataframe(sp500_income_statements, columns_to_drop, 10)
sp500_balance_sheets = pd.read_csv("sp500_balance_sheets.csv")
balance = clean_sp500_dataframe(sp500_balance_sheets, columns_to_drop_balance, 10)
sp500_cashflow_statements = pd.read_csv("sp500_cashflow_statements.csv")
text = "Gain Loss On Investment Securities	Short Term Debt Payments	Short Term Debt Issuance	Provisionand Write Offof Assets	Issuance Of Capital Stock	Common Stock Issuance	Change In Other Current Liabilities	Unrealized Gain Loss On Investment Securities	Gain Loss On Sale Of PPE	Earnings Losses From Equity Investments	Other Cash Adjustment Inside Changein Cash	Net Preferred Stock Issuance	Preferred Stock Issuance	Net Foreign Currency Exchange Gain Loss	Amortization Of Securities	Net Intangibles Purchase And Sale	Purchase Of Intangibles	Cash From Discontinued Financing Activities	Cash From Discontinued Investing Activities	Cash From Discontinued Operating Activities	Preferred Stock Dividend Paid	Dividend Received Cfo	Preferred Stock Payments	Net Investment Properties Purchase And Sale	Sale Of Investment Properties	Purchase Of Investment Properties	Other Cash Adjustment Outside Changein Cash	Change In Interest Payable	Dividends Received Cfi	Sale Of Intangibles	Excess Tax Benefit From Stock Based Compensation	Interest Received Cfi	Cash Flow From Discontinued Operation	Depletion	Interest Paid Cff	Taxes Refund Paid	Cash Flowsfromusedin Operating Activities Direct	Classesof Cash Payments	Other Cash Paymentsfrom Operating Activities	Paymentson Behalfof Employees	Classesof Cash Receiptsfrom Operating Activities	Other Cash Receiptsfrom Operating Activities	Dividend Paid Cfo"
cashflow = clean_sp500_dataframe(sp500_cashflow_statements, text.split("	"), 10)

In [54]:
def build_combined_quarterly(dfs: list[pd.DataFrame]) -> pd.DataFrame:
    # Each df in “dfs” must have columns ["Ticker", "Period", ...fundamentals...]
    indexed = []
    for df in dfs:
        temp = df.copy()
        temp = temp.set_index(["Ticker", "Period"], drop=False)
        indexed.append(temp)
    # Start from the first
    combined = indexed[0]
    for other in indexed[1:]:
        # Drop "Ticker" and "Period" from `other` so we don't duplicate those columns
        combined = combined.join(
            other.drop(columns=["Ticker", "Period"]),
            how="outer",
            lsuffix="",
            rsuffix=""
        )
    # Bring "Ticker" and "Period" back into columns
    combined = combined.reset_index(drop=True)
    # Sort for merge_asof
    combined.sort_values(["Ticker", "Period"], inplace=True)
    return combined



In [ ]:
if "Statement" in income.columns:    
    income = income.drop(columns=["Statement"])
if "Statement" in cashflow.columns:
    cashflow = cashflow.drop(columns=["Statement"])
if "Statement" in balance.columns:
    balance = balance.drop(columns=["Statement"])

combined_quarterly = build_combined_quarterly([income, balance, cashflow])
# 1) Optional: keep only one row per (Ticker, Period) in the quarterly table
combined_quarterly = combined_quarterly.drop_duplicates(subset=["Ticker","Period"])
combined_quarterly.to_csv("sp500_combined_quarterly.csv", index=False)
#5) Take your daily price table (sp500_prices) and turn it into long form.
prices = sp500_prices.copy()
prices = prices.rename(columns={"Date": "Period"})
prices["Period"] = pd.to_datetime(prices["Period"])

# Drop any column that is completely NaN (e.g. maybe some tickers have no data)
nan_cols = prices.columns[prices.isna().all()].tolist()
prices.drop(columns=nan_cols, inplace=True)

# Remove "_Close" suffix so each price column exactly matches Ticker
prices.columns = prices.columns.str.replace("_Close", "", regex=False)

# Make “Ticker” a category and pivot from wide→long via .stack()
prices.set_index("Period", inplace=True)
prices_long = (
    prices
    .stack(dropna=False)           # multi-index: (Period, Ticker) → Price
    .reset_index()
    .rename(columns={0: "Price",        # rename the stacked‐value column
                     "level_1": "Ticker"}) 
)
print(prices_long.head())
# 0) identical dtype against; coerce & sort
for df in (prices_long, combined_quarterly):
    df["Period"] = pd.to_datetime(df["Period"], errors="coerce")
prices_long        = prices_long.sort_values(["Ticker", "Period"])
combined_quarterly = combined_quarterly.sort_values(["Ticker", "Period"])

# 1) build hierarchical indices
daily_idx   = prices_long.set_index(["Ticker", "Period"])
quarter_idx = combined_quarterly.set_index(["Ticker", "Period"])

# 2) left-join then forward-fill within each ticker
df_daily_fund = (
    daily_idx
      .join(quarter_idx, how="left")        # align on exact dates
      .groupby(level=0)                     # level 0 = Ticker
      .ffill()                              # carry last available quarter fwd
      .reset_index()                        # back to flat columns
)
# 4) (Optional) save to CSV
df_daily_fund.to_csv("sp500_daily_fundamentals.csv", index=False)

C:\Users\georg.DESKTOP-2FS9VF1\AppData\Local\Temp\ipykernel_17356\2980590172.py:26: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack(dropna=False)           # multi-index: (Period, Ticker) → Price


      Period Ticker       Price
0 2020-01-02    MMM  121.761093
1 2020-01-02    AOS   43.202148
2 2020-01-02    ABT   78.908424
3 2020-01-02   ABBV   70.934235
4 2020-01-02    ACN  193.477829


In [68]:
sp500_info = pd.read_csv("sp500_info.csv")
sp500_info["Symbol"]   = sp500_info["Symbol"].astype(str).str.upper()
df_daily_fund["Ticker"] = df_daily_fund["Ticker"].astype(str).str.upper()

# 3)  Merge
df_daily_fund = (
    df_daily_fund
      .merge(
          sp500_info,
          left_on="Ticker",          # daily-fundamental side
          right_on="Symbol",         # info side
          how="left",                # keep every trading day even if a ticker is missing from info
      )
      .drop(columns=["Symbol"])      # “Ticker” already covers it
)

# 4)  (Optional) re-order columns so company info sits together
company_cols = [
    "Security", "GICS Sector", "GICS Sub-Industry",
    "Headquarters Location", "Date added", "CIK", "Founded"
]
remaining_cols = [c for c in df_daily_fund.columns if c not in company_cols]
df_daily_fund = df_daily_fund[remaining_cols[:3] + company_cols + remaining_cols[3:]]

# 5)  Save or inspect
df_daily_fund.to_csv("sp500_daily_fundamentals_enriched.csv", index=False)
print(df_daily_fund.head())

  Ticker     Period      Price              Security  GICS Sector  \
0      A 2020-01-02  82.885246  Agilent Technologies  Health Care   
1      A 2020-01-03  81.554459  Agilent Technologies  Health Care   
2      A 2020-01-06  81.795540  Agilent Technologies  Health Care   
3      A 2020-01-07  82.046280  Agilent Technologies  Health Care   
4      A 2020-01-08  82.856331  Agilent Technologies  Health Care   

                GICS Sub-Industry    Headquarters Location  Date added  \
0  Life Sciences Tools & Services  Santa Clara, California  2000-06-05   
1  Life Sciences Tools & Services  Santa Clara, California  2000-06-05   
2  Life Sciences Tools & Services  Santa Clara, California  2000-06-05   
3  Life Sciences Tools & Services  Santa Clara, California  2000-06-05   
4  Life Sciences Tools & Services  Santa Clara, California  2000-06-05   

       CIK Founded  ...  Net Income From Continuing Operations  \
0  1090872    1999  ...                                    NaN   
1  10908